In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop

!whoami
!date

ndbs
Fri May 22 13:10:03 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

## 1. Define directories and load count space data

In [5]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix'
output_directory = base_directory + '/v6.0_prelim_maternal_iron_lri_bp/india_5_20'

In [6]:
data = vol.load_transformed_count_data(f'{output_directory}/count_data')
data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

## 2. Look at deaths table; check scenarios and stratification

*  No iron fortification
*  Stratification not updated to new group definitions

In [7]:
data['deaths'].head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,232,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,232,folic_acid_fortification_scale_up,0.0


In [8]:
data['deaths'].scenario.unique()

array(['baseline', 'folic_acid_fortification_scale_up',
       'vitamin_a_fortification_scale_up'], dtype=object)

In [9]:
data['deaths'].folic_acid_fortification_group.unique()

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [10]:
data['deaths'].vitamin_a_fortification_group.unique()

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

## 3. Look at births and births_with_ntds tables

In [12]:
data['births'].head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,29,baseline,327.0
1,2020,female,covered,live_births,29,baseline,5.0
2,2020,female,covered,live_births,29,baseline,80.0
3,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,233.0
4,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,4.0


In [13]:
data['births_with_ntd'].head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births_with_ntds,29,baseline,0.0
1,2020,female,covered,live_births_with_ntds,29,baseline,0.0
2,2020,female,covered,live_births_with_ntds,29,baseline,0.0
3,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0
4,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0


In [14]:
data['births'].scenario.unique()

array(['baseline', 'folic_acid_fortification_scale_up',
       'vitamin_a_fortification_scale_up'], dtype=object)

## 4. Check state person time and person time tables

In [15]:
data['state_person_time'].head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,232,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,232,folic_acid_fortification_scale_up,0.0


In [16]:
data['state_person_time'].cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

In [17]:
data['person_time'].head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,232,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,232,folic_acid_fortification_scale_up,0.0


## 5. Compute prevalences by age group

In [18]:
numerator_strata = ['year', 'age_group']
denominator_strata = ['year', 'age_group']
multiplier = 100 # Get prevalences as percents
broadcast_cols = ['cause'] # Broadcast over cause column in state person time

prevalence = lop.rate_or_ratio(data['state_person_time'],
                               data['person_time'],
                               numerator_strata,
                               denominator_strata,
                               multiplier,
                               broadcast_cols,
                              )
prevalence.head()

,year,age_group,input_draw,scenario,cause,value
0,2020,1_to_4,29,baseline,diarrheal_diseases,1.257451
1,2020,1_to_4,29,baseline,lower_respiratory_infections,0.281531
2,2020,1_to_4,29,baseline,measles,0.115051
3,2020,1_to_4,29,baseline,neural_tube_defects,0.050326
4,2020,1_to_4,29,baseline,susceptible_to_diarrheal_diseases,98.199246


## 6. Test some pandas functions

In [19]:
prevalence.stack().head(10)

0  year                        2020
   age_group                 1_to_4
   input_draw                    29
   scenario                baseline
   cause         diarrheal_diseases
   value                    1.25745
1  year                        2020
   age_group                 1_to_4
   input_draw                    29
   scenario                baseline
dtype: object

## 7. Test my new describe function

In [22]:
lop.describe(prevalence, percentiles=[0.025, 0.975]).head(20)

count  \
year age_group scenario                          cause                                                
2020 1_to_4    baseline                          diarrheal_diseases                            10.0   
                                                 lower_respiratory_infections                  10.0   
                                                 measles                                       10.0   
                                                 neural_tube_defects                           10.0   
                                                 susceptible_to_diarrheal_diseases             10.0   
                                                 susceptible_to_lower_respiratory_infections   10.0   
                                                 susceptible_to_measles                        10.0   
                                                 susceptible_to_neural_tube_defects            10.0   
                                                 susceptible_to_vitamin_a_deficiency           10.0   
                                                 vitamin_a_deficiency                          10.0   
               folic_acid_fortification_scale_up diarrheal_diseases                            10.0   
                                                 lower_respiratory_infections                  10.0   
                                                 measles                                       10.0   
                                                 neural_tube_defects                           10.0   
                                                 susceptible_to_diarrheal_diseases             10.0   
                                                 susceptible_to_lower_respiratory_infections   10.0   
                                                 susceptible_to_measles                        10.0   
                                                 susceptible_to_neural_tube_defects            10.0   
                                                 susceptible_to_vitamin_a_deficiency           10.0   
                                                 vitamin_a_deficiency                          10.0   

                                                                                                   mean  \
year age_group scenario                          cause                                                    
2020 1_to_4    baseline                          diarrheal_diseases                            1.245838   
                                                 lower_respiratory_infections                  0.242103   
                                                 measles                                       0.171337   
                                                 neural_tube_defects                           0.048008   
                                                 susceptible_to_diarrheal_diseases            98.211136   
                                                 susceptible_to_lower_respiratory_infections  99.214872   
                                                 susceptible_to_measles                       97.667094   
                                                 susceptible_to_neural_tube_defects           99.408967   
                                                 susceptible_to_vitamin_a_deficiency          74.937420   
                                                 vitamin_a_deficiency                         24.519554   
               folic_acid_fortification_scale_up diarrheal_diseases                            1.249571   
                                                 lower_respiratory_infections                  0.241958   
                                                 measles                                       0.169501   
                                                 neural_tube_defects                           0.048914   
                                                 susceptible_to_diarrheal_diseases            98.207327   
        

### Check some prevalence estimates to see whether they add to 100%

Well, they're low by about 0.453%, but that's probably ok.

In [23]:
# LRI + susceptible to LRI in baseline
74.930194+24.526703

99.456897

In [24]:
# susceptible to VAD + VAD in FA scale-up
0.242103+99.214872

99.456975

## 8. Filter to LRI and check LRI prevalence by age group

* Looks like we're much closer to GBD now that birth prevalence is included!

In [26]:
prevalence.query("cause == 'lower_respiratory_infections'").head()

,year,age_group,input_draw,scenario,cause,value
1,2020,1_to_4,29,baseline,lower_respiratory_infections,0.281531
11,2020,1_to_4,29,folic_acid_fortification_scale_up,lower_respiratory_infections,0.278936
21,2020,1_to_4,29,vitamin_a_fortification_scale_up,lower_respiratory_infections,0.275317
31,2020,1_to_4,232,baseline,lower_respiratory_infections,0.213828
41,2020,1_to_4,232,folic_acid_fortification_scale_up,lower_respiratory_infections,0.205031


In [27]:
lri_prevalence = prevalence.query("cause == 'lower_respiratory_infections'")
lri_prevalence.shape

(600, 6)

In [28]:
lop.describe(lri_prevalence)

count  \
year age_group      scenario                          cause                                 
2020 1_to_4         baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     early_neonatal baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     late_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     post_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
2021 1_to_4         baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     early_neonatal baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     late_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     post_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
2022 1_to_4         baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     early_neonatal baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     late_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     post_neonatal  baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
2023 1_to_4         baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   10.0   
     early_neonatal baseline                          lower_respiratory_infections   10.0   
                    folic_acid_fortification_scale_up lower_respiratory_infections   10.0   
                    vitamin_a_fortification_scale_up  lower_respiratory_infections   

## 9. Compute change in prevalence from baseline to interventions

In [49]:
delta_prevalence = lop.averted(prevalence, 'baseline')
delta_prevalence.head()

,year,age_group,input_draw,cause,scenario,relative_to,value
0,2020,1_to_4,29,diarrheal_diseases,folic_acid_fortification_scale_up,baseline,-0.019993
1,2020,1_to_4,29,diarrheal_diseases,vitamin_a_fortification_scale_up,baseline,-0.005764
2,2020,1_to_4,29,lower_respiratory_infections,folic_acid_fortification_scale_up,baseline,0.002595
3,2020,1_to_4,29,lower_respiratory_infections,vitamin_a_fortification_scale_up,baseline,0.006214
4,2020,1_to_4,29,measles,folic_acid_fortification_scale_up,baseline,-0.000868


### Test some python/pandas stuff

In [41]:
prevalence.columns

Index(['year', 'age_group', 'input_draw', 'scenario', 'cause', 'value'], dtype='object')

In [42]:
['year', 'age_group', 'input_draw', 'scenario', 'cause', 'value'].index

<function list.index>

In [44]:
['year', 'age_group', 'input_draw', 'scenario', 'cause', 'value'].index('scenario')

3

In [45]:
prevalence.columns.get_loc('scenario')

3

## 10. Summarize the change in prevalence and take a look

Use pandas cross sections ([DataFrame.xs()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html#pandas.DataFrame.xs)) to select one cause at a time.

In [50]:
delta_prevalence_summary = lop.describe(delta_prevalence)
delta_prevalence_summary.head()

count  \
year age_group cause                        scenario                          relative_to          
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline      10.0   
                                            vitamin_a_fortification_scale_up  baseline      10.0   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline      10.0   
                                            vitamin_a_fortification_scale_up  baseline      10.0   
               measles                      folic_acid_fortification_scale_up baseline      10.0   

                                                                                               mean  \
year age_group cause                        scenario                          relative_to             
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline    -0.003733   
                                            vitamin_a_fortification_scale_up  baseline     0.000195   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline     0.000145   
                                            vitamin_a_fortification_scale_up  baseline     0.002012   
               measles                      folic_acid_fortification_scale_up baseline     0.001837   

                                                                                                std  \
year age_group cause                        scenario                          relative_to             
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline     0.011922   
                                            vitamin_a_fortification_scale_up  baseline     0.008045   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline     0.006005   
                                            vitamin_a_fortification_scale_up  baseline     0.005512   
               measles                      folic_acid_fortification_scale_up baseline     0.004705   

                                                                                                min  \
year age_group cause                        scenario                          relative_to             
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline    -0.019993   
                                            vitamin_a_fortification_scale_up  baseline    -0.013481   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline    -0.010737   
                                            vitamin_a_fortification_scale_up  baseline    -0.009546   
               measles                      folic_acid_fortification_scale_up baseline    -0.003909   

                                                                                                25%  \
year age_group cause                        scenario                          relative_to             
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline    -0.011877   
                                            vitamin_a_fortification_scale_up  baseline    -0.004511   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline    -0.002961   
                                            vitamin_a_fortification_scale_up  baseline    -0.000182   
               measles                      folic_acid_fortification_scale_up baseline    -0.001822   

                                                                                                50%  \
year age_group cause                        scenario                          relative_to             
2020 1_to_4    diarrheal_diseases           folic_acid_fortification_scale_up baseline    -0.006019   
                                            vitamin_a_fortification_scale_up  baseline     0.001034   
               lower_respiratory_infections folic_acid_fortification_scale_up baseline    

In [51]:
delta_prevalence_summary.xs('lower_respiratory_infections', level='cause')

count  \
year age_group      scenario                          relative_to          
2020 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2021 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2022 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2023 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2024 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   

                                                                       mean  \
year age_group      scenario                          relative_to             
2020 1_to_4         folic_acid_fortification_scale_up baseline     0.000145   
                    vitamin_a_fortification_scale_up  baseline     0.002012   
     early_neonatal folic_acid_fortification_scale_up baseline     0.026338   
                    vitamin_a_fortification_scale_up  baseline     0.031325   
     late_neonatal  folic_acid_fortification_scale_up baseline     0.040262   
                    vitamin_a_fortification_scale_up  baseline     0.035562   
     post_neonatal  folic_acid_fortification_scale_up baseline     0.002832   
                    vitamin_a_fortification_scale_up  baseline    -0.002164   
2021 1_to_4         folic_acid_fortification_scale_up baseline     0.001295   
     

In [52]:
delta_prevalence_summary.xs('vitamin_a_deficiency', level='cause')

count  \
year age_group      scenario                          relative_to          
2020 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2021 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2022 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2023 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
2024 1_to_4         folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     early_neonatal folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     late_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   
     post_neonatal  folic_acid_fortification_scale_up baseline      10.0   
                    vitamin_a_fortification_scale_up  baseline      10.0   

                                                                        mean  \
year age_group      scenario                          relative_to              
2020 1_to_4         folic_acid_fortification_scale_up baseline     -0.007149   
                    vitamin_a_fortification_scale_up  baseline     -0.013531   
     early_neonatal folic_acid_fortification_scale_up baseline     -0.074395   
                    vitamin_a_fortification_scale_up  baseline     -0.053485   
     late_neonatal  folic_acid_fortification_scale_up baseline     -0.033823   
                    vitamin_a_fortification_scale_up  baseline     -0.022617   
     post_neonatal  folic_acid_fortification_scale_up baseline     -0.008304   
                    vitamin_a_fortification_scale_up  baseline      0.011155   
2021 1_to_4         folic_acid_fortification_scale_up baseline      0.0019